     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-py3-none-any.whl size=1014850 sha256=772033f2bf79df13b8a4f13e133086dec07c0402cb67fbcb1532be263d8fa649
  Stored in directory: /root/.cache/pip/wheels/bd/b7/17/d60d18a7b3420135dfe0ce04d361b396f32b66b75a40471bb6
Successfully built dash-renderer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=4377f0a3e9b080f854799abdd97ee5a86611901cb39c

#Import Required libraries

In [ ]:
!pip install dash
!pip install dash-core-components
!pip install dash-html-components
!pip install dash-renderer
!pip install pyngrok
!pip install --upgrade pyngrok

!pip install dash dash-core-components dash-html-components dash-renderer plotly pyngrok


In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pickle
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pyngrok import ngrok
from dash import dcc, html

# Making Layout using dash

In [30]:
with open('lr_model.pkl', 'rb') as file:
    logistic_model = pickle.load(file)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Churn Prediction App', style={'textAlign': 'center', 'marginBottom': '20px'}),
    html.Label('Age'),
    dcc.Input(id='age', type='number', value=25, style={'marginRight': '10px'}),
    html.Label('Service Length (months)'),
    dcc.Input(id='service_length', type='number', value=10),
    html.Label('Monthly Charges (USD)'),
    dcc.Input(id='monthly_charges', type='number', value=100),
    html.Label('Total Charges (USD)'),
    dcc.Input(id='total_charges', type='number', value=500),
    html.Label('Contract Type'),
    dcc.Dropdown(
        id='contract_type',
        options=[
            {'label': 'Month-to-Month', 'value': 'Month-to-Month'},
            {'label': 'One-Year', 'value': 'One-Year'},
            {'label': 'Two-Year', 'value': 'Two-Year'}
        ],
        value='Month-to-Month'
    ),
    html.Label('Gender'),
    dcc.Dropdown(
        id='gender',
        options=[
            {'label': 'Male', 'value': 'Male'},
            {'label': 'Female', 'value': 'Female'}
        ],
        value='Male'
    ),
    html.Button('Submit', id='submit-button', n_clicks=0, style={'marginTop': '10px' , 'color': 'black','background-color':'white'}),
    html.H3(id='churn-prediction-output', style={'marginTop': '20px', 'fontWeight': 'bold'})
])

@app.callback(
    Output('churn-prediction-output', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('age', 'value'),
     State('service_length', 'value'),
     State('monthly_charges', 'value'),
     State('total_charges', 'value'),
     State('contract_type', 'value'),
     State('gender', 'value')]
)


def churn_prediction(n_clicks, age, service_length, monthly_charges, total_charges, contract_type, gender):
    if n_clicks is None or n_clicks == 0:
        return ''

    data = pd.read_csv("CommLink_Telecom_Customer_Data.csv")
    # Convert ServiceLength and TotalCharges columns to integer data type
    data['MonthlyCharges'] = data['MonthlyCharges'].astype(int)
    data['TotalCharges'] = data['TotalCharges'].astype(int)
    label_encoder = LabelEncoder()
    data['Gender'] = label_encoder.fit_transform(data['Gender'])
    data['ContractType'] = label_encoder.fit_transform(data['ContractType'])
    # Initialize MinMaxScaler
    scaler = MinMaxScaler()
    # Columns to scale
    numerical_columns = ['Age', 'ServiceLength', 'MonthlyCharges', 'TotalCharges']

     # Scale the numerical columns
    data[numerical_columns] = scaler.fit_transform(data[numerical_columns])


    X = data.drop(['CustomerID', 'Churn'], axis=1)
    y = data['Churn']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    churn_prediction = logistic_model.predict(X_train)
    if churn_prediction[0] == 0:
        churn_result = 'No'
    else:
        churn_result = 'Yes'

    return f'Churn Prediction: {churn_result}'


In [31]:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>